In [90]:
import yfinance as yf
from datetime import datetime, timedelta
import pandas as pd
import requests
import QuantLib as ql
import numpy as np
import matplotlib.pyplot as plt
import json
from sklearn.metrics import mean_squared_error
import scipy.optimize as opt
from scipy.optimize import differential_evolution
import concurrent.futures
from tradingview_screener import Query
import rookiepy


with open("config.json", "r") as config_file:
    config = json.load(config_file)

api_key = config.get("api_key")
secret_key = config.get("secret_key")

pd.set_option('display.max_rows', None)


In [59]:
NASDAQ = pd.read_csv('Indexes/NASDAQ.csv')
DOWJ = pd.read_csv('Indexes/DOWJ.csv')
SP = pd.read_csv('Indexes/S&P500.csv')

def clean_data(df):
    df = df[['Company', 'Symbol']]
    df = pd.DataFrame(df).dropna()
    return df

NASDAQ = clean_data(NASDAQ)
DOWJ = clean_data(DOWJ)
SP = clean_data(SP)


In [103]:
    

#TODO FIX SCREENER 
def screen_stocks():
    # Get cookies for TradingView session
    cookies = rookiepy.to_cookiejar(rookiepy.chrome(['.tradingview.com']))
    
    # Fetch stock data
    _, df = Query().select('close','change', 'EMA30').limit(1000).get_scanner_data(cookies=cookies)
    
    # Apply filters
    filtered_df = df[(df['change'] < -2.5) & (df['EMA30'] > 2) & df['close'] < 55]

    return filtered_df

def filter_stocks(): 
    filtered_stocks = set()
    # for index, stock in SP.iterrows():
    #     today_change, rolling_avg = get_price_change_and_rolling_avg(stock['Symbol'])
    #     if today_change is not None and rolling_avg is not None:  # Only process if valid data is returned
    #         if (today_change <= -2.00) and (rolling_avg > 0.00) and (get_current_stock_price(stock['Symbol']) < 150): 
    #             filtered_stocks.add(stock['Symbol'])

    for index, stock in DOWJ.iterrows():
        today_change, rolling_avg = get_price_change_and_rolling_avg(stock['Symbol'])
        if today_change is not None and rolling_avg is not None:
            if (today_change <= -2.00) and (rolling_avg > 0.00) and (get_current_stock_price(stock['Symbol']) < 150): 
                filtered_stocks.add(stock['Symbol'])

    for index, stock in NASDAQ.iterrows():
        today_change, rolling_avg = get_price_change_and_rolling_avg(stock['Symbol'])
        if today_change is not None and rolling_avg is not None:
            if (today_change <= -2.00) and (rolling_avg > 0.00) and (get_current_stock_price(stock['Symbol']) < 150): 
                filtered_stocks.add(stock['Symbol'])

    return filtered_stocks

def get_price_change_and_rolling_avg(ticker: str):
    try:
        # Define the date range
        end_date = datetime.now()
        start_date = end_date - timedelta(days=24)
        
        # Download stock data
        data = yf.download(ticker, start=start_date, end=end_date, progress=False)
        
        # If data is empty, return None
        if data.empty:
            return None, None
        
        data = data.sort_index()

        # Calculate the daily price change
        data['Price_Change'] = (data['Close'].diff() / data['Close'].shift(1)) * 100
        today_price_change = data['Price_Change'].iloc[-1]

        # Calculate the rolling average of the price change
        rolling_avg = data['Price_Change'].rolling(window=min(14, len(data))).mean().iloc[-1]

        return today_price_change, rolling_avg
    
    except Exception as e:
        print(f"Error occurred for ticker {ticker}: {e}")
        return None, None

def get_current_stock_price(symbol: str):

    url = "https://data.alpaca.markets/v2/stocks/trades/latest"

    headers = {
        "accept": "application/json",
        "APCA-API-KEY-ID": api_key,
        "APCA-API-SECRET-KEY": secret_key,
    }

    params = {
        "symbols": symbol,  
        "feed": "iex" 
    }

    try:
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()  

        data = response.json()
        return data.get("trades", {}).get(symbol, {}).get("p") 

    except requests.exceptions.RequestException as e:
        print(f"Error fetching stock price: {e}")


def fetch_option_chain(api_key: str, secret_key: str, underlying_symbol: str):

    url = f"https://data.alpaca.markets/v1beta1/options/snapshots/{underlying_symbol}?feed=indicative&limit=100&expiration_date=2025-02-07"
    headers = {
        "accept": "application/json",
        "APCA-API-KEY-ID": api_key,
        "APCA-API-SECRET-KEY": secret_key,
    }
    try:
        response = requests.get(url, headers=headers)
        data = response.json()
        snapshots = data.get('snapshots', {})
        return snapshots
    except requests.exceptions.RequestException as e:
        print(f"Error fetching option chain: {e}")
        return None

def parse_option_chain(option_chain):

    parsed_data = []
    
    for symbol, details in option_chain.items():
        expiration_start = len(symbol) - 15
        ticker = symbol[:expiration_start]
        expiration_date = f"20{symbol[expiration_start:expiration_start+2]}-{symbol[expiration_start+2:expiration_start+4]}-{symbol[expiration_start+4:expiration_start+6]}"
        option_type = "Call" if symbol[expiration_start+6] == "C" else "Put"
        strike_price = int(symbol[expiration_start+7:]) / 1000  
        
        greeks = details.get("greeks", {}) or {}
        implied_volatility = details.get("impliedVolatility", None)
        latest_quote = details.get("latestQuote", {})
        
        parsed_data.append({
            "ticker": ticker,
            "expiration_date": expiration_date,
            "option_type": option_type,
            "strike_price": strike_price,
            "delta": greeks.get("delta"),
            "gamma": greeks.get("gamma"),
            "rho": greeks.get("rho"),
            "theta": greeks.get("theta"),
            "vega": greeks.get("vega"),
            "implied_volatility": implied_volatility,
            "ask_price": latest_quote.get("ap"),
            "ask_size": latest_quote.get("as"),
            "bid_price": latest_quote.get("bp"),
            "bid_size": latest_quote.get("bs")
        })
    
    return pd.DataFrame(parsed_data)

def get_option_chain(api_key: str, secret_key: str, underlying_symbol: str):
    option_chain = fetch_option_chain(api_key, secret_key, underlying_symbol)
    if option_chain:
        return parse_option_chain(option_chain)
    else:
        return None
    

def gbm(s0, mu, sigma, deltaT, dt):
    """
    Models a stock price S(t) using the Wiener process W(t) as
    `S(t) = S(0).exp{(mu-(sigma^2/2).t)+sigma.W(t)}`
    
    Arguments:
        s0: Initial stock price, default 100
        mu: 'Drift' of the stock (upwards or downwards), default 0.2
        sigma: 'Volatility' of the stock, default 0.68
        deltaT: The time period for which the future prices are computed, default 52 (as in 52 weeks)
        dt: The granularity of the time-period, default 0.1
    
    Returns:
        time_vector: array of time steps
        s: array with the simulated stock prices over the time-period deltaT
    """
    n_step = int(deltaT / dt)  # Number of time steps
    time_vector = np.linspace(0, deltaT, num=n_step)  # Time vector
    
    # Wiener process: cumulative sum of random normal increments
    random_increments = np.random.normal(0, np.sqrt(dt), size=n_step)
    weiner_process = np.cumsum(random_increments)
    
    # Stock price simulation
    stock_var = (mu - (sigma**2 / 2)) * time_vector
    s = s0 * np.exp(stock_var + sigma * weiner_process)
    
    return s


def objective(params, real_prices, s0):
    """Objective function for optimization."""
    mu, sigma = params  # Unpack parameters
    gbm_prices = gbm(s0, mu, sigma, deltaT=len(real_prices), dt=1)
    return mean_squared_error(real_prices, gbm_prices)


def archive_objective(params, real_prices, s0):
    """Objective function for optimization."""
    mu, sigma = params  # Unpack parameters
    gbm_prices = gbm(s0, mu, sigma, deltaT=len(real_prices), dt=1)
    return mean_squared_error(real_prices, gbm_prices)

def archive_optimize_gbm(symbol):
    """
    Optimize μ and σ over multiple time bins, weighting recent periods more.
    """
    # Fetch real stock data (past 5 years)
    stock_data = yf.download(symbol, period="2y", interval="1d")
    real_prices = stock_data["Close"].dropna().values

    # Split into bins of 20 trading days
    bin_size = 20
    num_bins = len(real_prices) // bin_size
    weights = np.linspace(1, 2, num_bins)  # Increasing weights for recent bins

    mu_values, sigma_values, mses = [], [], []

    # Optimize each bin
    for i in range(num_bins):
        bin_prices = real_prices[i * bin_size : (i + 1) * bin_size]
        s0 = bin_prices[0]

        # Define the bounds for optimization
        bounds = [(-0.3, 0.3), (0.001, 0.35)]

        # Run the optimizer
        result = differential_evolution(objective, bounds, args=(bin_prices, s0))
        best_mu, best_sigma = result.x
        best_mse = result.fun

        mu_values.append(best_mu)
        sigma_values.append(best_sigma)
        mses.append(best_mse)

        print(f"Bin {i+1}/{num_bins}: μ = {best_mu:.4f}, σ = {best_sigma:.4f}, MSE = {best_mse:.4f}")

    # Compute weighted averages
    weight_sum = np.sum(weights)
    avg_mu = np.sum(np.array(mu_values) * weights) / weight_sum
    avg_sigma = np.sum(np.array(sigma_values) * weights) / weight_sum

    print(f"\nFinal Weighted Averages: μ = {avg_mu:.4f}, σ = {avg_sigma:.4f}")

    return avg_mu, avg_sigma

def gbm_vs_real_graph(symbol, mu, sigma, period):
    stock_data = yf.download(symbol, period=period, interval="1d")
    real_prices = stock_data["Close"].dropna().values
    time_steps = np.arange(len(real_prices))


    gbm_path = gbm(s0 = real_prices[0], mu=mu, sigma=sigma, deltaT=len(real_prices), dt=1)
    plt.figure(figsize=(10, 5))
    plt.plot(time_steps, real_prices, label="Real Prices", color="blue")
    plt.plot(time_steps, gbm_path, label="GBM Simulated", linestyle="dashed", color="red")
    
    plt.xlabel("Time (Days)")
    plt.ylabel("Price")
    plt.title(f"GBM vs Real Prices for {symbol}")
    plt.legend()
    plt.grid()
    plt.show()

def compute_score(contract):
    """Compute the weighted score for a contract."""
    return (0.25 * contract['profitability_percent'] +
            0.25 * contract['percent_return'] +
            0.50 * contract['sharpe_ratio'])

def select_optimal_contract(contracts):
    """Select the contract with the highest weighted score."""
    # Compute the score for each contract
    contracts['score'] = contracts.apply(compute_score, axis=1)
    
    # Find the contract with the maximum score
    contracts = contracts.sort_values(by='score', ascending=False)
    optimal_contract = contracts.loc[contracts['score'].idxmax()]
    
    return contracts

# Example usage
# avg_mu, avg_sigma = optimize_gbm("AAPL")
# gbm_vs_real_graph('AAPL', mu=0.0011, sigma=0.0130, period='2y')

In [75]:
candidates = filter_stocks()
simulation_attempts = 100
# t-bill 3-month rate: 4.19%, inflation rate: 2.9% -> scaled to weekly
risk_free_rate = (((1 + 0.0419) / (1 + 0.029)) ** (1/52) - 1) * 100
all_options = pd.DataFrame(columns=['ticker', 'expiration_date', 'option_type', 'strike_price', 'delta', 'gamma', 'rho', 'theta', 'vega', 'implied_volatility', 'ask_price', 'ask_size', 'bid_price', 'bid_size'])
print(candidates)

for symbol in candidates:
    option_chain = get_option_chain(api_key, secret_key, symbol)
    put_chain = option_chain[(option_chain['option_type'] == 'Put') & (option_chain['rho'].notna())].sort_values(by='strike_price', ascending=True)

    if option_chain is None or option_chain.empty:
        continue 

    price = get_current_stock_price(symbol)
    optimized_mu, optimized_sigma = archive_optimize_gbm(symbol)

    profitability_chances = []
    percent_returns = []

    for index, contract in put_chain.iterrows():
        count = 0
        strike_price = contract['strike_price']

        for i in range(simulation_attempts):
            prices = gbm(s0=price, mu=optimized_mu, sigma=optimized_sigma, 
                deltaT=np.busday_count(datetime.today().date(), datetime.strptime(contract['expiration_date'], "%Y-%m-%d").date()), dt=1)
        
            if prices[-1] > strike_price:
                count += 1

        profitability_chance = (count / simulation_attempts) * 100
        profit = (contract['bid_price']*contract['bid_size'] + contract['ask_price']*contract['ask_size']) / (contract['ask_size'] + contract['bid_size'])
        percent_return = (profit / (strike_price)) * 100

        profitability_chances.append(profitability_chance)
        percent_returns.append(percent_return)

    put_chain['profitability_percent'] = profitability_chances
    put_chain['percent_return'] = percent_returns
    put_chain['expected_value'] = put_chain['profitability_percent'] * put_chain['percent_return']
    put_chain['current_price'] = price

    if put_chain['percent_return'].std() != 0:
        put_chain['sharpe_ratio'] = (put_chain['percent_return'] - risk_free_rate) / put_chain['percent_return'].std()
    else:
        put_chain['sharpe_ratio'] = 0  # Avoid division by zero

    all_options = pd.concat([all_options, put_chain], ignore_index=True, copy=False)



{'WBD', 'PDD'}


[*********************100%***********************]  1 of 1 completed


Bin 1/25: μ = -0.0022, σ = 0.0137, MSE = 0.0771
Bin 2/25: μ = 0.0053, σ = 0.0149, MSE = 0.0801
Bin 3/25: μ = -0.0072, σ = 0.0094, MSE = 0.0879
Bin 4/25: μ = -0.0041, σ = 0.0117, MSE = 0.0557
Bin 5/25: μ = -0.0041, σ = 0.0377, MSE = 0.1385
Bin 6/25: μ = -0.0037, σ = 0.0224, MSE = 0.0435
Bin 7/25: μ = -0.0015, σ = 0.0244, MSE = 0.0901
Bin 8/25: μ = -0.0084, σ = 0.0261, MSE = 0.0607
Bin 9/25: μ = 0.0002, σ = 0.0132, MSE = 0.0291
Bin 10/25: μ = 0.0022, σ = 0.0284, MSE = 0.1919
Bin 11/25: μ = 0.0079, σ = 0.0081, MSE = 0.0507
Bin 12/25: μ = -0.0059, σ = 0.0135, MSE = 0.0188
Bin 13/25: μ = -0.0016, σ = 0.0087, MSE = 0.0216
Bin 14/25: μ = -0.0082, σ = 0.0168, MSE = 0.0516
Bin 15/25: μ = -0.0001, σ = 0.0054, MSE = 0.0153
Bin 16/25: μ = -0.0013, σ = 0.0206, MSE = 0.0228
Bin 17/25: μ = -0.0040, σ = 0.0147, MSE = 0.0102
Bin 18/25: μ = 0.0003, σ = 0.0083, MSE = 0.0055
Bin 19/25: μ = 0.0039, σ = 0.0426, MSE = 0.0689
Bin 20/25: μ = -0.0002, σ = 0.0353, MSE = 0.0197
Bin 21/25: μ = -0.0035, σ = 0.0463,

/var/folders/90/x6spqd2j4cbb38yrckqjm8840000gn/T/ipykernel_17550/2435079775.py:56: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_options = pd.concat([all_options, put_chain], ignore_index=True, copy=False)
[*********************100%***********************]  1 of 1 completed


Bin 1/25: μ = 0.0008, σ = 0.0185, MSE = 6.1783
Bin 2/25: μ = -0.0103, σ = 0.0215, MSE = 9.0802
Bin 3/25: μ = -0.0056, σ = 0.0061, MSE = 1.5371
Bin 4/25: μ = 0.0053, σ = 0.0156, MSE = 4.2328
Bin 5/25: μ = 0.0002, σ = 0.0343, MSE = 3.2157
Bin 6/25: μ = 0.0075, σ = 0.0202, MSE = 4.0626
Bin 7/25: μ = -0.0072, σ = 0.0079, MSE = 1.5140
Bin 8/25: μ = 0.0150, σ = 0.0423, MSE = 11.0108
Bin 9/25: μ = 0.0093, σ = 0.0333, MSE = 3.2060
Bin 10/25: μ = 0.0021, σ = 0.0205, MSE = 1.8955
Bin 11/25: μ = 0.0183, σ = 0.0305, MSE = 14.3381
Bin 12/25: μ = 0.0004, σ = 0.0096, MSE = 1.9404
Bin 13/25: μ = -0.0016, σ = 0.0235, MSE = 9.5841
Bin 14/25: μ = -0.0007, σ = 0.0258, MSE = 6.4999
Bin 15/25: μ = -0.0036, σ = 0.0154, MSE = 4.3281
Bin 16/25: μ = 0.0109, σ = 0.0101, MSE = 9.5104
Bin 17/25: μ = 0.0017, σ = 0.0102, MSE = 4.1886
Bin 18/25: μ = -0.0071, σ = 0.0121, MSE = 3.9890
Bin 19/25: μ = -0.0015, σ = 0.0195, MSE = 4.4597
Bin 20/25: μ = -0.0187, σ = 0.0471, MSE = 48.3306
Bin 21/25: μ = 0.0249, σ = 0.0224, MS

In [84]:
# TODO
"""
- Figure out way to normalize stock price, whether that is min max of the range of the price(shoudl help optimizer
- Find better metric for optimizer
- Try binning, so like get past 10 years of AAPL, seperate into bins of 20 or n trading days, train optimzer on each one. Then the hyperparameters can be weighted to have more bias towards more recent bins
"""

contracts = select_optimal_contract(all_options).iloc[:5]
display(contracts)


,ticker,expiration_date,option_type,strike_price,delta,gamma,rho,theta,vega,implied_volatility,ask_price,ask_size,bid_price,bid_size,profitability_percent,percent_return,expected_return,current_price,sharpe_ratio,score
10,PDD,2025-02-07,Put,97.0,-0.1034,0.0240,-0.0012,-0.1669,0.0199,0.6760,0.39,250,0.37,11,100.0,0.401193,40.119287,105.26,0.218101,25.209349
8,PDD,2025-02-07,Put,95.0,-0.0747,0.0174,-0.0009,-0.1423,0.0156,0.7331,0.32,161,0.24,20,100.0,0.327537,32.753707,105.26,0.175516,25.169642
5,PDD,2025-02-07,Put,92.0,-0.0542,0.0116,-0.0007,-0.1295,0.0122,0.8555,0.33,348,0.12,236,100.0,0.266453,26.645325,105.26,0.140199,25.136713
7,PDD,2025-02-07,Put,94.0,-0.0582,0.0142,-0.0007,-0.1181,0.0129,0.7378,0.22,1,0.20,37,100.0,0.213326,21.332587,105.26,0.109483,25.108073
6,PDD,2025-02-07,Put,93.0,-0.0504,0.0122,-0.0006,-0.1098,0.0115,0.7693,0.20,33,0.17,158,100.0,0.188369,18.836908,105.26,0.095054,25.094619


In [104]:
stonks = screen_stocks()

display(stonks)


,ticker,close,change,EMA30
0,NASDAQ:NVDA,116.6600,-2.840010,133.435237
1,AMEX:SPY,597.7700,-0.672959,597.551329
2,NASDAQ:TSLA,383.6800,-5.170539,401.073214
3,NASDAQ:QQQ,518.1100,-0.800322,519.326990
4,NASDAQ:AAPL,228.0100,-3.385593,236.203131
5,NASDAQ:META,697.4600,1.201428,634.189678
6,NASDAQ:PLTR,83.7400,1.515335,74.982037
7,NASDAQ:MSFT,410.9200,-0.997446,429.498535
8,NASDAQ:AMZN,237.4200,-0.109391,227.863414
9,AMEX:IWM,223.8300,-1.170081,226.194460


In [63]:
# Archive

# def multithread_optimize_bin(bin_prices, bin_size, weights, i):
#     s0 = bin_prices[0]

#     # Define the bounds for optimization
#     bounds = [(-0.3, 0.3), (0.001, 0.30)]

#     # Run the optimizer for the bin
#     result = differential_evolution(objective, bounds, args=(bin_prices, s0))
#     best_mu, best_sigma = result.x
#     best_mse = result.fun

#     print(f"Bin {i+1}: μ = {best_mu:.4f}, σ = {best_sigma:.4f}, MSE = {best_mse:.4f}")
#     return best_mu, best_sigma, best_mse

# def multithread_optimize_gbm(symbol): 
    """
    Optimize μ and σ over multiple time bins, weighting recent periods more.
    """
    # Fetch real stock data (past 2 years)
    stock_data = yf.download(symbol, period="2y", interval="1d")
    real_prices = stock_data["Close"].dropna().values

    # Split into bins of 20 trading days
    bin_size = 20
    num_bins = len(real_prices) // bin_size
    weights = np.linspace(1, 2, num_bins)  # Increasing weights for recent bins

    # Initialize containers for results
    mu_values, sigma_values, mses = [], [], []

    # Use concurrent.futures for parallel processing of bins
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = []
        for i in range(num_bins):
            bin_prices = real_prices[i * bin_size : (i + 1) * bin_size]
            futures.append(executor.submit(optimize_bin, bin_prices, bin_size, weights, i))
        
        for future in concurrent.futures.as_completed(futures):
            best_mu, best_sigma, best_mse = future.result()
            mu_values.append(best_mu)
            sigma_values.append(best_sigma)
            mses.append(best_mse)

    # Compute weighted averages
    weight_sum = np.sum(weights)
    avg_mu = np.sum(np.array(mu_values) * weights) / weight_sum
    avg_sigma = np.sum(np.array(sigma_values) * weights) / weight_sum

    print(f"\nFinal Weighted Averages: μ = {avg_mu:.4f}, σ = {avg_sigma:.4f}")

    return avg_mu, avg_sigma